# Model
$\frac{d N_{ia}}{dt}   = R \cdot f_{ia}(N_{ia}(t), N_{ib}(t), N_{sab}(t), X_b(t)) \cdot (1 - d\cdot N(t)/K) \cdot N_{ia}(t) - m \cdot N_{ia}(t)$

$\frac{d N_{ib}}{dt}   = R \cdot f_{ib}(N_{ia}(t), N_{ib}(t), N_{sab}(t), X_b(t)) \cdot (1 - d\cdot N(t)/K) \cdot 
N_{ib}(t) - m \cdot N_{ib}(t)$

$\frac{d N_{sab}}{dt}   = R \cdot f_{sab}(N_{ia}(t), N_{ib}(t), N_{sab}(t), X_b(t)) \cdot (1 - d\cdot N(t)/K) \cdot N_{sab}(t) - m \cdot N_{sab}(t)$


I chose $f_{ia}, f_{ib}$ and $f_{sab}$ as the quantity of food a player gets by choosing the corresponding strategy:

$f_{ia}(N_{ia}(t), N_{ib}(t), N_{sab}(t), X_b(t)) = 
\left\{
\begin{array}{lll}
\frac{1}{N_{ia}(t) + N_{sab}(t)} & if & X_b = l \\
\frac{1}{N_{ia}(t)} & if & X_b = h \\
\end{array}
\right.$ 

$f_{ib}(N_{ia}(t), N_{ib}(t), N_{sab}(t), X_b(t)) = 
\left\{
\begin{array}{lll}
\frac{l}{N_{ib}(t)} & if & X_b = l \\
\frac{h}{N_{ib}(t) +  N_{sab}(t)} & if & X_b = h \\
\end{array}
\right.$ 

$f_{sab}(N_{ia}(t), N_{ib}(t), N_{sab}(t), X_b(t)) = 
\left\{
\begin{array}{lll}
\frac{1}{N_{ia}(t) + N_{sab}(t)} & if & X_b = l \\
\frac{h}{N_{ib}(t) + N_{sab}(t)} & if & X_b = h \\
\end{array}
\right.$ 

On the following simulation, we can see that each frequency always converges to the theoretical prediction.

# Simulation

In [5]:
from dashboard import *
Dashboard()

Dashboard(children=(HBox(children=(Box(children=(Image_box(children=(Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x…

### If the simulation doesn't work (it may take a long time)
1. refresh
2. click again on show widget